In [93]:
import numpy as np
import cv2

In [94]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [95]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    alpha=K[0,0]
    beta=K[1,1]
    c_x=K[0,2]
    c_y=K[1,2]
    
    pc= np.zeros((depth.shape[0],depth.shape[1],3)) # x,y,z
    y_index= np.arange(0,depth.shape[0])[:,None]
    x_index= np.arange(0,depth.shape[1])[None,:]
    pc[:, :, 0] = (x_index-c_x)/alpha*depth
    pc[:, :, 1] = (y_index-c_y)/beta*depth
    pc[:, :, 2] = depth
    
    mask=seg==255
    pc = pc[mask].reshape(-1,3)
    print(pc.shape)
    # ------------TODO --------------
    return pc

partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

(18705, 3)


In [96]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]
    
partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

# -----------TODO---------------
# implement one way chamfer distance
N=2048
one_way_CD = 0
partial_pc_sampled = partial_pc_sampled[None,:,:]
full_pc_sampled = full_pc_sampled[:,None,:]
one_way_CD = np.sum(np.min(np.linalg.norm(partial_pc_sampled-full_pc_sampled,axis=2),axis=0))/N
one_way_CD = one_way_CD[None]
# -----------TODO---------------
print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', one_way_CD)

one way chamfer distance:  [0.00997117]


one way chamfer distance:  0.009976257336995639
